# mcp_exchange_rates.py server with brave search 
# This is a demo do not use for personal trading decisions
## to get current fx and exchange rate trading recommendations
## prequisites
### 1. uses claude-sonnet-4
ANTHROPIC_API_KEY in .env file
### 2. use Brave Search API key
BRAVE_API in .env file




In [ ]:
# import libraries
import os
import asyncio
from re import I
from agents import Agent, OpenAIChatCompletionsModel, Runner, trace
from agents.mcp import MCPServerStdio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display


In [ ]:
load_dotenv(override=True)

# 1. Configure Claude client
claude_client = AsyncOpenAI(
    base_url="https://api.anthropic.com/v1",
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

# 2. Create Claude model
claude_model = OpenAIChatCompletionsModel(
    model="claude-sonnet-4-20250514",
    openai_client=claude_client
)

# 3. MCP server parameters (Brave Search)
search_params = {
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-brave-search"],
    "env": {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
}
# 4. MCP server parameters (Forex)
exchange_rate_params = {
    "command": "uv", "args": ["run", "mcp_exchange_rates.py"]
}

In [ ]:
print(exchange_rate_params)
print(search_params)

In [ ]:


try:
    #5. Connect to MCP server
    async with MCPServerStdio(
        params=exchange_rate_params, client_session_timeout_seconds=30) as forex_server:
        print("✅ Connected to Forex MCP server")
       # 6. Connect to MCP server
        async with MCPServerStdio(
            params=search_params, 
            client_session_timeout_seconds=60
        ) as search_server:
            
            print("✅ Connected to Brave Search MCP server")
            
            # 7. List available tools
            tools = await search_server.list_tools()
            forex_tools = await forex_server.list_tools()
            tools = tools + forex_tools
            print(f"�� Available tools: {tools}")
            # 8. instructions and request
            instructions="""You are a financial research assistant. You have access to web search tools and usd conversion tools.
                Use the search tools to find current information and provide accurate, up-to-date responses.
                Always cite your sources when providing information."""
            
            request ="you have USD 100000 to investment and want to make a profit. provide the latest exchange rate of \
                        usd/aud usd/eur, usd/nzd, usd/gbp, usd/can - print the rates in a table format\
                        print the inverse rates table of the above rates\
                        include the source and date of the information\
                        for each currency pair based get the latest news about the summary of the current market conditions for each currency pair. \
                        provide citations for source of news/information. \
                        and \
                        the current exchange rates recommend trades for usd/aud, usd/eur, usd/nzd, usd/gbp, usd/can \
                        provide a trade idea based on the news and the current market conditions"
            # 9. Create Claude agent with MCP server
            agent = Agent(
                name="claude_researcher",
                instructions=instructions,
                model=claude_model,
                mcp_servers=[search_server,forex_server]
            )
            
            print("✅ Claude agent created with MCP servers")
            
            # 10. Use the agent with tracing
            with trace("Claude and Forex Web Research"):
                result = await Runner.run(
                    agent,
                    request,
                    max_turns=25
                    )
            
            print("\n📊 Claude Agent Response:")
            print("=" * 50)
            # print(result.final_output)
            display(Markdown("This is a demo do not use for personal trading decisions"))
            display(Markdown(result.final_output))
            print("=" * 50)
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

